In [ ]:
import random
import pandas as pd
import numpy as np
from itertools import combinations
import scipy.stats as stats

###**Part 1: Fisher’s mode of inference of Randomized Experiments**<br/>

1. In children at risk for malaria, research is conducted in stimulating the immune system with
micronutrients supplementation, an important one being vitamin A (e.g., Shankar et al., Lancet 1999, 354:203-9). Table 1 gives measurements of the malaria parasite Plasmodium falciparum (Pf), in $log_e$ (counts / microL), for 6 children taking vitamin A supplementation and 6 children taking a control (placebo) treatment.

    We wish to test Fisher’s (sharp) null hypothesis of no treatment effect on these 12 children. Take the test statistic to be the difference in sample means. Assume that the assignment in Table 1 was a completely randomized experiment

<center>
  <caption> Table 1: Parasite load [$log_e$(count/microL)].</caption>
  <table>
    <tr>
      <th>
        Placebo
      </th>
      <th>
        Vitamin A
      </th>
    </tr>
    <tr>
      <td> 8.62 </td>
      <td> 0.06 </td> 
    </tr>
    <tr>
      <td> 1.48 </td>
      <td> 1.72 </td> 
    </tr>
    <tr>
      <td> 8.93 </td>
      <td> 2.19 </td> 
    </tr>
    <tr>
      <td> 9.57 </td>
      <td> 7.37 </td> 
    </tr>
    <tr>
      <td> 2.65 </td>
      <td> 7.53 </td> 
    </tr>
    <tr>
      <td> 7.30 </td>
      <td> 7.62 </td> 
    </tr>
  </table>
</center>



In [ ]:
control = [8.62, 1.48, 8.93, 9.57, 2.65, 7.30]
treatment = [0.06, 1.72, 2.19, 7.32, 7.53, 7.62]
all = np.array(control+treatment).reshape((12, 1))

(a) Calculate the exact p–value (two-tailed). [Hint: assuming the null hypothesis, write a program
to find the 924 possible values of the test statistic under the assignment mechanism.]

In [ ]:
treatment_idxs = list(combinations(range(12), 6))
assigment_operator = np.array([[1 if idx in treatment else 0 for idx in range(12)] for treatment in treatment_idxs][::-1], dtype = int)

In [ ]:
treatment_scores = (assigment_operator@all)/6
control_scores = ((1-assigment_operator)@all)/6
test_statistic = np.round(treatment_scores - control_scores, 2)

df = pd.DataFrame(assigment_operator, dtype = int, columns = ['Z'+str(i) for i in range(1, 13)] )
df['levels'] = test_statistic

df.head()

,Z1,Z2,Z3,Z4,Z5,Z6,Z7,Z8,Z9,Z10,Z11,Z12,levels
0,0,0,0,0,0,0,1,1,1,1,1,1,-2.02
1,0,0,0,0,0,1,0,1,1,1,1,1,0.40
2,0,0,0,0,0,1,1,0,1,1,1,1,-0.16
3,0,0,0,0,0,1,1,1,0,1,1,1,-0.31
4,0,0,0,0,0,1,1,1,1,0,1,1,-2.03


In [ ]:
T_obs = df.loc[0, 'levels']
round(2*sum(df.levels<=T_obs)/len(df),2)

0.27

(b) Report the p–value obtained by drawing 1, 000 times from the distribution of the statistic
under the null hypothesis and comparing the draws with the observed statistic.

In [ ]:
treatment_idxs = np.random.randint(0, 12, size = (1000, 6))
treatmemt_assignment = np.array([[1 if idx in treatment else 0 for idx in range(12)] for treatment in treatment_idxs][::-1], dtype = int)
control_idxs = np.random.randint(0, 12, size = (1000, 6))
control_assignment = np.array([[1 if idx in treatment else 0 for idx in range(12)] for treatment in control_idxs][::-1], dtype = int)

In [ ]:
treatment_scores = (treatmemt_assignment@all)/6
control_scores = (control_assignment@all)/6
test_statistic = np.round(treatment_scores - control_scores, 2)

df = pd.DataFrame(treatmemt_assignment, dtype = int, columns = ['Z'+str(i) for i in range(1, 13)] )
df['levels'] = test_statistic
test_statistic.shape
df.head()

,Z1,Z2,Z3,Z4,Z5,Z6,Z7,Z8,Z9,Z10,Z11,Z12,levels
0,1,0,0,0,1,0,0,0,0,0,1,1,-0.37
1,0,0,1,0,0,1,0,1,1,1,0,0,2.60
2,1,0,0,0,0,1,1,0,0,0,1,1,0.38
3,0,1,1,0,0,0,0,0,1,0,1,1,2.83
4,1,1,0,0,1,0,0,0,1,0,0,0,-2.40


In [ ]:
round(2*sum(df.levels<=T_obs)/len(df),2)

0.27

(c)  Report the p–value by using a t-test.

In [ ]:
np.var(control)/np.var(treatment)

1.014428019594967

In [ ]:
stats.ttest_ind(a=control, b=treatment, equal_var=True)

Ttest_indResult(statistic=1.0089695865082862, pvalue=0.33677915183055496)

Thus, we can assume the variance is equal. 

(d) The potential outcome framework has three key components: (i) potential outcomes, (ii) assignment mechanism, (iii) a (probabilistic) model for the potential outcomes. In the above
questions, (a) returns an exact p-value, both (b) and (c) are making some approximations of (a). So in terms of what component of the potential outcome framework that (b) approximates
(a) and (c) approximates (a)?

**ANSWER**: 
(b) approximates (a) and (c) approximates (a) in terms of the assignment mechanism. 

2. Now assume that prior to the experiment, researchers used a characteristic X (e.g., baseline severity measures) of the children to match them in 6 pairs (the 6 pairs being the 6 rows of Table 1),
and then they assigned treatment at random within each pair. Answer 1(a)-1(d) for the pairwise
assignment (For (c) use a paired t-test)

In [ ]:
pairs = [(i, i+6) for i in range(6)]

a. Calculate the exact p–value (two-tailed).


In [ ]:
treatment_idxs = []
def solve(pairs, idxs = []):
  if len(idxs) == 6:
    treatment_idxs.append(idxs)
    return 
  solve(pairs[1:], idxs + [pairs[0][0]])
  solve(pairs[1:], idxs + [pairs[0][1]])

solve(pairs) 

In [ ]:
assigment_operator = np.array([[1 if idx in treatment else 0 for idx in range(12)][::-1] for treatment in treatment_idxs], dtype = int)

In [ ]:
treatment_scores = (assigment_operator@all)/6
control_scores = ((1-assigment_operator)@all)/6
test_statistic = np.round(treatment_scores - control_scores, 2)

df = pd.DataFrame(assigment_operator, dtype = int, columns = ['Z'+str(i) for i in range(1, 13)] )
df['levels'] = test_statistic

df.head()

,Z1,Z2,Z3,Z4,Z5,Z6,Z7,Z8,Z9,Z10,Z11,Z12,levels
0,0,0,0,0,0,0,1,1,1,1,1,1,-2.02
1,1,0,0,0,0,0,0,1,1,1,1,1,0.83
2,0,1,0,0,0,0,1,0,1,1,1,1,-2.10
3,1,1,0,0,0,0,0,0,1,1,1,1,0.76
4,0,0,1,0,0,0,1,1,0,1,1,1,0.23


In [ ]:
T_obs = df.loc[0, 'levels']
round(2*sum(df.levels<=T_obs)/len(df),2)

0.38

(b) Report the p–value obtained by drawing 1, 000 times from the distribution of the statistic
under the null hypothesis and comparing the draws with the observed statistic.


In [ ]:
repeated_treatment_idxs = [treatment_idxs[random.randint(0, 63)] for _ in range(1000)]
assigment_operator = np.array([[1 if idx in treatment else 0 for idx in range(12)] for treatment in repeated_treatment_idxs][::-1], dtype = int)

In [ ]:
treatment_scores = (assigment_operator@all)/6
control_scores = ((1-assigment_operator)@all)/6
test_statistic = np.round(treatment_scores - control_scores, 2)

df = pd.DataFrame(assigment_operator, dtype = int, columns = ['Z'+str(i) for i in range(1, 13)] )
df['levels'] = test_statistic

df.head()

,Z1,Z2,Z3,Z4,Z5,Z6,Z7,Z8,Z9,Z10,Z11,Z12,levels
0,1,1,0,1,1,0,0,0,1,0,0,1,-0.12
1,1,0,1,1,0,0,0,1,0,0,1,1,3.83
2,1,0,0,1,0,1,0,1,1,0,1,0,1.48
3,0,1,0,1,0,1,1,0,1,0,1,0,-1.46
4,0,0,0,1,1,1,1,1,1,0,0,0,-3.00


In [ ]:
round(2*sum(df.levels<=T_obs)/len(df),2)

0.37

(c) Report the p–value by using a t-test.

In [ ]:
stats.ttest_rel(control, treatment)

Ttest_relResult(statistic=0.995586226461929, pvalue=0.36516098315767737)

3. By writing the test statistic (difference in sample means) as an explicit function of the potential
outcomes and the assignment indicators, show that, under complete randomization, the statistic is
unbiased for the average treatment effect. Do the same for the pairwise randomization.

Done by hand!



4.  In a study with complete randomization, is it possible for the data to provide evidence that the
treatment effect is (or not) additive, i.e. $Yi(1) = Yi(0) + k$ for some fixed $k$ all $i$? If not,
explain. If yes, describe briefly how you would use a statistical method to look for evidence
against additivity, for example, in the context of the data in Table 1.



Yes. 

$$Y_i(1) = Y_i(0) + k $$
$$Y_i(1) - Y_i(0) = k$$


Then, by setting the following hypothesis testing: 

$$H_0: k = 0$$
$$H_A: k = 0$$

We can use Fisher's null hypothesis test to check if there is evidence against additivity i.e., if the p value > 0.05, then we can accept the null hypothesis and say there is evidence against additivity. 

###**Part 2: Neyman’s mode of inference of Randomized Experiments**<br/>

Assume a targeted population has N = 12 units, with the potential outcomes under control and treatment
given in Table 2.

<center>
<caption> Table 2: </caption>
  <table>
    <tr>
      <th>
        Y(1)
      </th>
      <th>
        Y(0)
      </th>
    </tr>
    <tr>
      <td> 35 </td>
      <td> 40 </td> 
    </tr>
    <tr>
      <td> 45 </td>
      <td> 55 </td> 
    </tr>
    <tr>
      <td> 55 </td>
      <td> 55 </td> 
    </tr>
    <tr>
      <td> 65 </td>
      <td> 70 </td> 
    </tr>
    <tr>
      <td> 25 </td>
      <td> 30 </td> 
    </tr>
    <tr>
      <td> 45 </td>
      <td> 55 </td> 
    </tr>
    <tr>
      <td> 60 </td>
      <td> 65 </td> 
    </tr>
    <tr>
      <td> 75 </td>
      <td> 80 </td> 
    </tr>
    <tr>
      <td> 35 </td>
      <td> 40 </td> 
    </tr>
    <tr>
      <td> 55 </td>
      <td> 50 </td> 
    </tr>
    <tr>
      <td> 35 </td>
      <td> 40 </td> 
    </tr>
    <tr>
      <td> 65 </td>
      <td> 70 </td> 
    </tr>
  </table>
</center>


In [ ]:
treatment = np.array([35, 45, 55, 65, 25, 45, 60, 75, 35, 55, 35, 65])
control = np.array([40, 55, 55, 70, 30, 55, 65, 80, 40, 50, 40, 70])
N0, N1, N = 2, 2, len(treatment)

1. Verify the following formula via simulation, as described below:
$$Var[\tilde{Y}(1) - \tilde{Y}(0) | \textbf{Y}(1), \textbf{Y}(0)] = \frac{\sigma_0^2}{N_0} + \frac{\sigma_1^2}{N_1} -\frac{\sigma_{01}^2}{N}  $$
,
where $\sigma_z = \frac{1}{N-1}\sum_{i=1}^N (Y_i - \bar{Y}(z))^2$ is the population variance of $Y(z)$ for $z = 0, 1$, and $\sigma_{0,1} = \frac{1}{N-1} 
\sum_{i=1}^N(Y_i(1) − Y_i(0) − (\bar{Y}(1) − \bar{Y}(0)))^2$ 
is the population variance of the individual treatment effects (the variance of $Y_i(1) − Y_i(0)$ across the population), $N$ is the total population size, and $N_1$ is the size of the treatment group chosen and $N_0$ is the size of the control group chosen in a completely randomized experiment.

   To verify this formula using simulation, use the 12 observations in Table 2 as the population. For each sample of size 4 out of the 12 units in Table 2 (there will be $\binom {12}{4} = 495$ such samples), randomly assign 2 out of the 4 units (there will be $\binom {4}{2} = 6$ possible random assignment for each
sample of size 4), and compute all possible values of $\bar{Y}_{obs}(1) - \bar{Y}_{obs}(0)$ of these 6 randomizations.

   Then calculate the variance of these $495 \times 6$ values of $\bar{Y}_{obs}(1) - \bar{Y}_{obs}(0)$, and confirm the formula given above.

#### Calculate the Variance Using the Formula Above

In [ ]:
s2_1 = sum((treatment-treatment.mean())**2)/(N - 1)
s2_0 = sum((control-control.mean())**2)/(N - 1)
s2_01 = sum((treatment - control - (treatment.mean() - control.mean()))**2)/(N-1)

s2_0/N0 + s2_1/N1 - s2_01/N

228.8983585858586

#### Verify the Formula Using the Simulated Version

In [ ]:
# get the indicies of the 4 bootstrapped samples 
sampled_idxs = list(combinations(range(12), N0+N1))
samples = np.array([[1 if idx in treatment else 0 for idx in range(12)][::-1] for treatment in sampled_idxs], dtype = int)

# get the treatment samples
treatment_samples = samples*treatment
treatment_samples = treatment_samples[treatment_samples>0].reshape((-1, 4))

# get the control samples
control_samples = samples*control
control_samples = control_samples[control_samples>0].reshape((-1, 4))

## treatment assignment operator 
treatment_idxs = list(combinations(range(N0+N1), N1))
assigment_operator = np.array([[1 if idx in treatment else 0 for idx in range(N0+N1)] for treatment in treatment_idxs][::-1], dtype = int)

# get the y_treatment and y_control
y_obs_treatment = (treatment_samples@assigment_operator.T)/2
y_obs_control = (control_samples@(1-assigment_operator).T)/2

#get the variance of mean(Y(1)) - mean(Y(0))
np.var(y_obs_treatment-y_obs_control)

228.89835858585857

Since, we get the same results in the simulated version and the in the formula, we can tell that the formula results are valid! 

2. . Verify that the usual (Neyman) estimator of the variance $\frac{S_0^2}{N_0}+\frac{S_1^2}{N_1}$ overestimates the true variance $Var[\bar{Y}(1) - \bar{Y}(0)|\textbf{Y(1)}, \textbf{Y(0)}]$.  To calculate $E\left( \frac{S_0^2}{N_0}+\frac{S_1^2}{N_1}|\textbf{Y(1)}, \textbf{Y(0)}\right)$, calculate the value
of $\frac{S_0^2}{N_0}+\frac{S_1^2}{N_1}$ for each of the 495 × 6 possible samples and randomizations, and take the mean. You should find that this is larger than the value of the variance found above, as additivity does not
hold here.

In [ ]:
s2_0/N0 + s2_1/N1

230.20833333333334

In [ ]:
y_obs_treatment_control = []
for idx in range(495):
  treatment_instance = treatment_samples[idx]*assigment_operator
  treatment_instance = treatment_instance[treatment_instance>0].reshape((-1, 2))
  y_obs_treatment = np.sum((treatment_instance-treatment_instance.mean(axis = 1).reshape((-1,1)))**2, axis = 1)/2

  control_instance = control_samples[idx]*assigment_operator
  control_instance = control_instance[control_instance>0].reshape((-1, 2))
  y_obs_control = np.sum((control_instance-control_instance.mean(axis = 1).reshape((-1,1)))**2, axis = 1)/2
  y_obs_treatment_control.append(y_obs_treatment+y_obs_control)

np.mean(y_obs_treatment_control)

230.20833333333334